# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# Libraries
import numpy as np
import pandas as pd

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [2]:
# your code here
table = pd.read_csv(r'/Users/telmabatista/Documents/Ironhack/Labs/lab-advanced-pandas/your-code/data/Admission_Predict.csv')

table

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,384,312,103,3,3.5,4.0,8.78,0,0.67


Remove trailing spaces at the end of the column names if there are any.


In [3]:
new_columns = [x.strip() for x in list(table.columns)]

table = table.set_axis(new_columns, axis = 1)

table

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,384,312,103,3,3.5,4.0,8.78,0,0.67


Let's evaluate the dataset by looking at the `head` function.

In [4]:
# your code here
table.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [5]:
# your code here
table.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [6]:
# your code here
table.set_index('Serial No.')

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...
381,324,110,3,3.5,3.5,9.04,1,0.82
382,325,107,3,3.0,3.5,9.11,1,0.84
383,330,116,4,5.0,4.5,9.45,1,0.91


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [7]:
# your code here

table["GRE_CGPA"] = table["GRE Score"].map(str) + '_' + table["CGPA"].map(str)

table["GRE_CGPA"].nunique()

385

Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [8]:
# your code here

table.set_index("GRE_CGPA", inplace = True)
table.drop(columns = ["GRE Score","CGPA"])

table.reset_index(inplace = True)

table


,GRE_CGPA,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,337_9.65,1,337,118,4,4.5,4.5,9.65,1,0.92
1,316_8.0,2,316,104,3,3.0,3.5,8.00,1,0.72
2,322_8.67,3,322,110,3,3.5,2.5,8.67,1,0.80
3,314_8.21,4,314,103,2,2.0,3.0,8.21,0,0.65
4,330_9.34,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...,...
380,324_9.04,381,324,110,3,3.5,3.5,9.04,1,0.82
381,325_9.11,382,325,107,3,3.0,3.5,9.11,1,0.84
382,330_9.45,383,330,116,4,5.0,4.5,9.45,1,0.91
383,312_8.78,384,312,103,3,3.5,4.0,8.78,0,0.67


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [9]:
#importing again since it had previous instructions to delete columns used ahead
admissions = pd.read_csv(r'/Users/telmabatista/Documents/Ironhack/Labs/lab-advanced-pandas/your-code/data/Admission_Predict.csv')
new_columns = [x.strip() for x in list(admissions.columns)]
admissions = admissions.set_axis(new_columns, axis = 1)
admissions

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,384,312,103,3,3.5,4.0,8.78,0,0.67


In [10]:
# your code here
df1 = admissions[(admissions.CGPA > 9) & (admissions.Research == 1)]
answer1 = df1.count()

answer1

Serial No.           101
GRE Score            101
TOEFL Score          101
University Rating    101
SOP                  101
LOR                  101
CGPA                 101
Research             101
Chance of Admit      101
dtype: int64

Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [11]:
# your code here
df2 = admissions[(admissions.CGPA > 9) & (admissions.SOP < 3.5)]
df2

df2["Chance of Admit"].mean()

0.8019999999999999

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [12]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    
    # your code here
    col=list(col)
    average =np.mean(col)
    std_dev =np.std(col)
    return [(x-average)/std_dev for x in col]

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [13]:
# your code here
CGPA_std = standardize(admissions["CGPA"])
admissions["CGPA_std"] = CGPA_std

GRE_std = standardize(admissions["GRE Score"])
admissions["GRE_std"] = GRE_std

LOR_std = standardize(admissions["LOR"])
admissions["LOR_std"] = LOR_std

admissions

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197
1,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840
2,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338
4,5,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338
...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,0.736216,0.629546,0.076840
381,382,325,107,3,3.0,3.5,9.11,1,0.84,0.852571,0.716170,0.076840
382,383,330,116,4,5.0,4.5,9.45,1,0.91,1.417729,1.149292,1.193197
383,384,312,103,3,3.5,4.0,8.78,0,0.67,0.304036,-0.409947,0.635019


We will generate the decision choice at random using the code below. Please run the cell.

In [14]:
# Libraries
from random import choices

In [15]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=admissions.shape[0])
len(decision_choice)

385

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [16]:
# your code here
deciding_column = admissions.lookup(admissions.index, decision_choice)
admissions["deciding_column"] = deciding_column
 

Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [17]:
# your code here
decision = [1 if x > 0.8 else 0 for x in admissions["deciding_column"]]
admissions["decision"] = decision

How many applicants will be accepted to the program using the decision column? Compute the result below.

In [18]:
# your code here

admissions[(admissions.decision == 1)].count()



Serial No.           87
GRE Score            87
TOEFL Score          87
University Rating    87
SOP                  87
LOR                  87
CGPA                 87
Research             87
Chance of Admit      87
CGPA_std             87
GRE_std              87
LOR_std              87
deciding_column      87
decision             87
dtype: int64

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [19]:
#importing again 
admissions = pd.read_csv(r'/Users/telmabatista/Documents/Ironhack/Labs/lab-advanced-pandas/your-code/data/Admission_Predict.csv')
admissions
list(admissions.columns)

['Serial No.',
 'GRE Score',
 'TOEFL Score',
 'University Rating',
 'SOP',
 'LOR ',
 'CGPA',
 'Research',
 'Chance of Admit ']

In [28]:
# your code here
admissions.columns = [x.strip().lower().replace(".","").replace(" ","_") for x in list(admissions.columns)]
admissions

,serial_no,gre_score,toefl_score,university_rating,sop,lor,cgpa,research,chance_of_admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,384,312,103,3,3.5,4.0,8.78,0,0.67


In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [ ]:
admissions["adjusted_gre"] = pd.cut(admissions.apply(lambda x: x["GRE Score"]+10 if x["University Rating"]> 4 else x["GRE Score"], axis=1),4, labels=[1,2,3,4])
admissions